<p style="align: center;">
    <img align=center src="../img/dls_logo.jpg" width=500 height=500>
</p>

<h1 style="text-align: center;">
    Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ
</h1>

---

In [ ]:
# !pip install -q catalyst

In [ ]:
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import TensorDataset, DataLoader

from catalyst import utils

sns.set(style='darkgrid', font_scale=1.4)

## Logistic regression is my profession

**Напоминание.** Давайте вспоним, что происходит в логистической регрессии. На входе у нас есть матрица объекты-признаки $X$ и вектор-столбец $y$ – метки из $\{0, 1\}$ для каждого объекта. Мы хотим найти такую матрицу весов $W$ и смещение $b$ (bias), что наша модель $XW + b$ будет каким-то образом предсказывать класс объекта. Как видно, на выходе наша модель может выдавать число в интервале от $(-\infty;\infty)$. Этот выход, как правило, называют **"логитами"** (**logits**). Нам необходимо перевести его на интервал $[0, 1]$ для того, чтобы он выдавал нам вероятность принадлежности объекта к классу один. Также лучше, чтобы эта функция была монотонной, быстро считалась, имела производную и на $-\infty$ имела значение $0$, а на $+\infty$ имела значение $1$. Такой класс функций называется сигмоиды. Чаще всего в качестве сигмоиды берут

$$
\sigma(x) = \frac{1}{1 + e^{-x}}
$$

### Задание

Вам необходимо написать модуль на **PyTorch** реализующий $logits = XW + b$, где $W$ и $b$ – параметры (`nn.Parameter`) модели. Иначе говоря здесь мы реализуем своими руками модуль `nn.Linear` (в этом пункте его использование запрещено). Инициализируйте веса нормальным распределением (`torch.randn`).

In [ ]:
class LinearRegression(nn.Module):
    def __init__(self, in_features: int, out_features: int, bias: bool = True):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(in_features, out_features))
        self.bias = bias
        if self.bias:
            self.bias_term = nn.Parameter(torch.randn(out_features, 1))

    def forward(self, x):
        x = x @ self.weights
        if self.bias:
            x += self.bias_term
        return x

### Датасет

Давайте сгенерируем датасет и посмотрим на него:

In [ ]:
X, y = make_moons(n_samples=10000, random_state=42, noise=0.1)

In [ ]:
plt.figure(figsize=(16, 10))
plt.title('Dataset')
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis')
plt.show()

Разделим на обучающую и тестовую части:

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)

### Загрузка данных

В **PyTorch** загрузка данных, как правило, происходит налету (иногда датасеты не помещаются в оперативную память). Для этого используются `Dataset` и `DataLoader`:

1. `Dataset` загружает каждый объект по отдельности.

2. `DataLoader` группирует объекты из `Dataset` в батчи.

Так как наш датасет достаточно маленький мы будем использовать `TensorDataset`. Всё, что нам нужно, это перевести данные из массива **NumPy** в тензор с типом `torch.float32`.

In [ ]:
X_train_t = torch.from_numpy(X_train).to(torch.float32)
y_train_t = torch.from_numpy(y_train).to(torch.float32)
X_val_t = torch.from_numpy(X_val).to(torch.float32)
y_val_t = torch.from_numpy(y_val).to(torch.float32)

Создаем `Dataset` и `DataLoader`:

In [ ]:
train_dataset = TensorDataset(X_train_t, y_train_t)
val_dataset = TensorDataset(X_val_t, y_val_t)
train_dataloader = DataLoader(train_dataset, batch_size=128)
val_dataloader = DataLoader(val_dataset, batch_size=128)

In [ ]:
utils.set_global_seed(42)
linear_regression = LinearRegression(2, 1)
loss_function = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(linear_regression.parameters(), lr=0.05)

---

#### Вопрос 1

Сколько обучаемых параметров у получившейся модели?

**Ответ:** $3$

In [ ]:
sum(p.numel() for p in linear_regression.parameters())

---

### Train loop

Вот псевдокод, который поможет вам разобраться в том, что происходит во время обучения:

```python
for epoch in range(max_epochs):  # <----------- итерируемся по датасету несколько раз
    for x_batch, y_batch in dataset:  # <------ итерируемся по датасету, так как мы используем 
                                      # <------ SGD а не GD, то берём батчи заданного размера
        optimizer.zero_grad()  # <------------- обуляем градиенты модели
        outp = model(x_batch)  # <------------- получаем "логиты" из модели
        loss = loss_func(outp, y_batch)  # <--- считаем "лосс" для логистической регрессии
        loss.backward()  # <------------------- считаем градиенты
        optimizer.step()  # <------------------ делаем шаг градиентного спуска
        if convergence:  # <------------------- в случае сходимости выходим из цикла
            break
```

В коде ниже добавлено логирование `accuracy` и `loss`.

In [ ]:
tol = 1e-3
losses = []
max_epochs = 100

prev_weights = torch.zeros_like(linear_regression.weights)
prev_bias_term = torch.zeros_like(linear_regression.bias_term)

stop_it = False

for epoch in range(max_epochs):
    
    utils.set_global_seed(42 + epoch)
    
    for it, (X_batch, y_batch) in enumerate(train_dataloader):
        optimizer.zero_grad()
        outp = linear_regression(X_batch)
        loss = loss_function(outp, y_batch.view(-1, 1))
        loss.backward()
        losses.append(loss.detach().flatten()[0])
        optimizer.step()
        probabilities = torch.sigmoid(outp)
        preds = (probabilities > 0.5).type(torch.long)
        batch_acc = (preds.flatten() == y_batch).type(torch.float32).sum() / y_batch.size(0)
        if it % 500000 == 0:
            print(f'Iteration: {it + epoch * len(train_dataset)}\nBatch accuracy: {batch_acc}')
        current_weights = linear_regression.weights.detach().clone()
        current_bias_term = linear_regression.bias_term.detach().clone()
        if (prev_weights - current_weights).abs().max() < tol and (prev_bias_term - current_bias_term).abs().max() < tol:
            print(f'\nIteration: {it + epoch * len(train_dataset)}. Convergence. Stopping iterations.')
            stop_it = True
            break
        prev_weights = current_weights
        prev_bias_term = current_bias_term
    
    if stop_it:
        break

---

#### Вопрос 2

Сколько итераций потребовалось, чтобы алгоритм сошелся?

**Ответ:** $105054$

In [ ]:
it + epoch * len(train_dataset)

---

### Визуализируем результаты

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(range(len(losses)), losses)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.show()

In [ ]:
import numpy as np

sns.set(style='white')

xx, yy = np.mgrid[-1.5:2.5:.01, -1.:1.5:.01]
grid = np.c_[xx.ravel(), yy.ravel()]
batch = torch.from_numpy(grid).type(torch.float32)

with torch.no_grad():
    probs = torch.sigmoid(linear_regression(batch).reshape(xx.shape))
    probs = probs.numpy().reshape(xx.shape)

f, ax = plt.subplots(figsize=(16, 10))
ax.set_title('Decision boundary', fontsize=14)
contour = ax.contourf(xx, yy, probs, 25, cmap='RdBu',
                      vmin=0, vmax=1)
ax_c = f.colorbar(contour)
ax_c.set_label('$P(y = 1)$')
ax_c.set_ticks([0, .25, .5, .75, 1])

ax.scatter(X[100:,0], X[100:, 1], c=y[100:], s=50,
           cmap='RdBu', vmin=-.2, vmax=1.2,
           edgecolor='white', linewidth=1)

ax.set(xlabel='$X_1$', ylabel='$X_2$')
plt.show()

---

#### Вопрос 3

Какое `accuracy` получается после обучения?

**Ответ:** $0.8688$

In [ ]:
@torch.no_grad()
def predict(dataloader, model):
    model.eval()
    predictions = np.array([])
    for x_batch, _ in dataloader:
        outp = model(x_batch)
        probs = torch.sigmoid(outp)
        preds = (probs > 0.5).type(torch.long)
        predictions = np.hstack((predictions, preds.numpy().flatten()))
    predictions = predictions
    return predictions.flatten()

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(predict(val_dataloader, linear_regression), y_val_t)

---

## MNIST

In [ ]:
from catalyst.contrib.datasets import MNIST

train_dataset = MNIST('data', train=True, download=True)
val_dataset = MNIST('data', train=False)
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
valid_dataloader = DataLoader(val_dataset, batch_size=128, shuffle=False)

In [ ]:
class Identical(nn.Module):
    def forward(self, x):
        return x

class Flatten(nn.Module):
    def forward(self, x):
        batch_size = x.size(0)
        return x.view(batch_size, -1)

In [ ]:
# activation = Identical
# activation = nn.ReLU
# activation = nn.LeakyReLU
activation = nn.ELU

model = nn.Sequential(
    Flatten(),
    nn.Linear(28 * 28, 128),
    activation(),
    nn.Linear(128, 128),
    activation(),
    nn.Linear(128, 10),
)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

loaders = {'train': train_dataloader, 'valid': valid_dataloader}

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

### Train loop

Давайте разберемся с кодом ниже, который подойдет для $90\%$ задач в будущем.

```python
for epoch in range(max_epochs):  # <----------------- итерируемся по датасету несколько раз
    for k, dataloader in loaders.items():  # <------- несколько dataloader для train / valid / test
        for x_batch, y_batch in dataloader:  # <----- итерируемся по датасету, так как мы используем
                                             # <----- SGD а не GD, то берём батчи заданного размера
            if k == "train":
                model.train()  # <------------------- переводим модель в режим train
                optimizer.zero_grad()  # <----------- обнуляем градиенты модели
                outp = model(x_batch)
                loss = criterion(outp, y_batch) # <-- считаем "лосс" для логистической регрессии
                loss.backward()  # <----------------- считаем градиенты
                optimizer.step()  # <---------------- делаем шаг градиентного спуска
            else:  # <------------------------------- test / eval
                model.eval()  # <-------------------- переводим модель в режим eval
                with torch.no_grad():  # <----------- НЕ считаем градиенты
                    outp = model(x_batch)  # <------- получаем "логиты" из модели
            count_metrics(outp, y_batch)  # <-------- считаем метрики
```

In [ ]:
max_epochs = 10

accuracy = {'train': [], 'valid': []}

for epoch in range(max_epochs):
    
    epoch_correct = 0
    epoch_all = 0
    utils.set_global_seed(42 + epoch)
    
    for k, dataloader in loaders.items():
        for x_batch, y_batch in dataloader:
            if k == 'train':
                model.train()
                optimizer.zero_grad()
                outp = model(x_batch.float())
                loss = criterion(outp, y_batch)
            else:
                model.eval()
                with torch.no_grad():
                    outp = model(x_batch.float())
            preds = outp.argmax(-1)
            correct = (preds == y_batch).sum()
            total = len(y_batch)
            epoch_correct += correct.item()
            epoch_all += total
            if k == 'train':
                loss = criterion(outp, y_batch)
                loss.backward()
                optimizer.step()
        if k == 'train':
            print(f'Epoch: {epoch + 1}')
        print(f'Loader: {k}. Accuracy: {epoch_correct / epoch_all}')
        accuracy[k].append(epoch_correct / epoch_all)

Попробуйте разные функции активации. Позапускайте ячейки сверху, несколько раз изменив функции активации. После каждого обучения запустите одну из ячеек снизу. Таким образом мы получим график `accuracy / epoch` для каждой функции активации.

In [ ]:
plain_accuracy = accuracy['valid']

In [ ]:
relu_accuracy = accuracy['valid']

In [ ]:
leaky_relu_accuracy = accuracy['valid']

In [ ]:
elu_accuracy = accuracy['valid']

### Accuracy

In [ ]:
sns.set(style='darkgrid', font_scale=1.4)

plt.figure(figsize=(16, 10))
plt.title('Valid accuracy')
plt.plot(range(max_epochs), plain_accuracy, label='No activation', linewidth=2)
plt.plot(range(max_epochs), relu_accuracy, label='ReLU activation', linewidth=2)
plt.plot(range(max_epochs), leaky_relu_accuracy, label='LeakyReLU activation', linewidth=2)
plt.plot(range(max_epochs), elu_accuracy, label='ELU activation', linewidth=2)
plt.legend()
plt.xlabel('Epoch')
plt.show()

---

#### Вопрос 4

Какая из активаций показала наивысший `accuracy`?

**Ответ:** авторы считают, что ReLU, но в реальности зависит от того, где запускать.

---

## Свёрточные слои

### Ядра

In [ ]:
import cv2

sns.set(style="white")

img = cv2.imread("data/mipt.jpg")
RGB_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(12, 8))
plt.imshow(RGB_img)
plt.show()

Попробуйте посмотреть, как различные свёртки влияют на фото. Например, попробуйте:

```python
[0, 0, 0],
[0, 1, 0],
[0, 0, 0]
```

```python
[0, 1, 0],
[0, -2, 0],
[0, 1, 0]
```

```python
[0, 0, 0],
[1, -2, 1],
[0, 0, 0]
```

```python
[0, 1, 0],
[1, -4, 1],
[0, 1, 0]
```

```python
[0, -1, 0],
[-1, 5, -1],
[0, -1, 0]
```

```python
[0.0625, 0.125, 0.0625],
[0.125, 0.25, 0.125],
[0.0625, 0.125, 0.0625]
```

Не стесняйтесь пробовать свои варианты!

In [ ]:
img_t = torch.from_numpy(RGB_img).type(torch.float32).unsqueeze(0)

# kernel = torch.tensor([
#     [0, 0, 0],
#     [0, 1, 0],
#     [0, 0, 0]
# ]).reshape(1, 1, 3, 3).type(torch.float32)

# kernel = torch.tensor([
#     [0, 1, 0],
#     [0, -2, 0],
#     [0, 1, 0]
# ]).reshape(1, 1, 3, 3).type(torch.float32)

# kernel = torch.tensor([
#     [0, 0, 0],
#     [1, -2, 1],
#     [0, 0, 0]
# ]).reshape(1, 1, 3, 3).type(torch.float32)

kernel = torch.tensor([
    [0, 1, 0],
    [1, -4, 1],
    [0, 1, 0]
]).reshape(1, 1, 3, 3).type(torch.float32)

# kernel = torch.tensor([
#     [0, -1, 0],
#     [-1, 5, -1],
#     [0, -1, 0]
# ]).reshape(1, 1, 3, 3).type(torch.float32)

# kernel = torch.tensor([
#     [0.0625, 0.125, 0.0625],
#     [0.125, 0.25, 0.125],
#     [0.0625, 0.125, 0.0625]
# ]).reshape(1, 1, 3, 3).type(torch.float32)

kernel = kernel.repeat(3, 3, 1, 1)
# [BS, H, W, C] -> [BS, C, H, W]
img_t = img_t.permute(0, 3, 1, 2)
# pad image for the same output size
img_t = nn.ReflectionPad2d(1)(img_t)

result = F.conv2d(img_t, kernel)[0]

In [ ]:
plt.figure(figsize=(12, 8))
result_np = result.permute(1, 2, 0).numpy() / 256 / 3

plt.imshow(result_np)
plt.show()

---

#### Вопрос 5

Как вы считаете, как можно описать действия ядер, приведенных выше? Сопоставьте для каждой буквы число.

1. Размытие

2. Увеличение резкости

3. Тождественное преобразование

4. Выделение вертикальных границ

5. Выделение горизонтальных границ

6. Выделение границ

**Ответ:** A - 3, Б - 5, В - 4, Г - 6, Д - 2, Е - 1.

---

### LeNet

Если мы сделаем параметры свёрток обучаемыми, то сможем добиться хороших результатов для задач компьютерного зрения. Давайте посмотрим на архитектуру, предложенную еще в $1998$ году!

In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square conv kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # 5x5 image resolution
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, int(x.nelement() / x.shape[0]))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
model = LeNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

loaders = {'train': train_dataloader, 'valid': valid_dataloader}

In [ ]:
max_epochs = 10

accuracy = {'train': [], 'valid': []}

for epoch in range(max_epochs):
    
    epoch_correct = 0
    epoch_all = 0
    utils.set_global_seed(42 + epoch)
    
    for k, dataloader in loaders.items():
        for x_batch, y_batch in dataloader:
            if k == 'train':
                model.train()
                optimizer.zero_grad()
                outp = model(x_batch.float().unsqueeze(1))
                loss = criterion(outp, y_batch)
            else:
                model.eval()
                with torch.no_grad():
                    outp = model(x_batch.float().unsqueeze(1))
            preds = outp.argmax(-1)
            correct = (preds == y_batch).sum()
            total = len(y_batch)
            epoch_correct += correct.item()
            epoch_all += total
            if k == 'train':
                loss = criterion(outp, y_batch)
                loss.backward()
                optimizer.step()
        if k == 'train':
            print(f'Epoch: {epoch + 1}')
        print(f'Loader: {k}. Accuracy: {epoch_correct / epoch_all}')
        accuracy[k].append(epoch_correct / epoch_all)

In [ ]:
lenet_accuracy = accuracy['valid']

Сравним с предыдущим пунктом:

In [ ]:
sns.set(style='darkgrid', font_scale=1.4)
plt.figure(figsize=(16, 10))
plt.title('Valid accuracy')
plt.plot(range(max_epochs), relu_accuracy, label='ReLU activation', linewidth=2)
plt.plot(range(max_epochs), leaky_relu_accuracy, label='LeakyReLU activation', linewidth=2)
plt.plot(range(max_epochs), elu_accuracy, label='ELU activation', linewidth=2)
plt.plot(range(max_epochs), lenet_accuracy, label='LeNet', linewidth=2)
plt.legend()
plt.xlabel('Epoch')
plt.show()

---

#### Вопрос 6

Какое `accuracy` получается после обучения с точностью до двух знаков после запятой?

**Ответ:**

In [ ]:
round(lenet_accuracy[-1], 2)